## LAB: La maldición de la dimensionalidad

Para entender el efecto de la cantidad de dimensiones en la capacidad predictiva del modelo vamos a trabajar en un clasificador de noticias de dos diarios argentinos para intentar distinguir entre uno y otro a partir del vocabulario que utilizan. <br />

Para esto vamos a implementar un modelo de tipo Naïve Bayes con vectorización de tipo TF-IDF.

In [1]:
import pandas as pd
df_clarin = pd.read_csv('../Data/clarin.csv')
df_clarin['class'] = 0

In [2]:
df_p12 = pd.read_csv('../Data/pagina12.csv')
df_p12['class'] = 1

In [3]:
df = pd.concat([df_clarin,df_p12])

In [4]:
df.sample(10)

,Unnamed: 0,cuerpo,fecha_hora,imagen,resumen,suplemento,titulo,url,class
267,267,El diagnóstico es crítico. El presente de Se...,10/10/2017,//images.clarin.com/2017/10/10/SJof9t53W_930x5...,El piloto de Ferrari ya sabe lo que es reverti...,/deportes/,Fórmula 1: Sebastian Vettel buscará una nueva ...,http://www.clarin.com/deportes/automovilismo/f...,0
216,216,"Lewis Hamilton, líder del campeonato de Fórm...",07/10/2017,//images.clarin.com/2017/10/07/Byqm2E8hW_930x5...,El británico largará primero en la carrera del...,/deportes/automovilismo/,Fórmula 1: Hamilton logró su primera pole en J...,http://www.clarin.com/deportes/automovilismo/f...,0
244,244,"Hijo de un padre maltratador, ladrón de auto...",21/09/2017,//images.clarin.com/2017/09/20/ryvDgGgi-_930x5...,Murió uno de los más polémicos y fascinantes d...,/deportes/boxeo/,LaMotta y una vida de película que mutó del bo...,http://www.clarin.com/deportes/boxeo/lamotta-v...,0
221,221,San Lorenzo (5-0) fue muy sólido y demolió a...,08/10/2017,//images.clarin.com/2017/10/08/HkUd-8O2Z_930x5...,El conjunto de Gonzalo García ganó como visita...,/deportes/basquet/,Súper 20: San Lorenzo aplastó a Ferro y sigue ...,http://www.clarin.com/deportes/basquet/liga-na...,0
27,27,Del 12 al 16 de octubre se realizará en Taipei...,09 de octubre de 2017,https://images.pagina12.com.ar/styles/width960...,NaN,NaN,Mundial de Kung Fu,https://www.pagina12.com.ar/67927-mundial-de-k...,1
194,194,"Un importante número de personas, aún sin dete...",10 de octubre de 2017,https://images.pagina12.com.ar/styles/focal_16...,NaN,El mundo,Los ultras de Valencia,https://www.pagina12.com.ar/68143-los-ultras-d...,1
314,314,"El martes comenzará 3 8 S M, una coproducció...",06/10/2017,//images.clarin.com/2017/10/05/B1fkpQVnb_930x5...,El director francés Laurent Berger dirige 23 8...,/espectaculos/teatro/,Una maratón sobre el más grande,http://www.clarin.com/espectaculos/teatro/mara...,0
113,113,Ante una nueva “versión periodística” lanzada ...,09 de octubre de 2017,https://images.pagina12.com.ar/styles/focal_16...,"A través de un comunicado, los familiares de S...",El país,"Otra operación, otra desmentida",https://www.pagina12.com.ar/68072-otra-operaci...,1
107,107,Se acerca diciembre y en la Legislatura de la ...,09 de octubre de 2017,https://images.pagina12.com.ar/styles/focal_16...,Temen que el Gobierno de la Ciudad consiga est...,Sociedad,Movilizados contra la construcción de un shopping,https://www.pagina12.com.ar/68050-movilizados-...,1
43,43,Para cerrar la II Feria del Libro Deportivo qu...,09 de octubre de 2017,https://images.pagina12.com.ar/styles/focal_16...,NaN,NaN,Deporte y derechos humanos,https://www.pagina12.com.ar/67911-deporte-y-de...,1


### 1. Limpieza

#### 1.1 Faltantes

A partir del dataset observamos que los campos que probablemente contengan el vocabulario relevante son "cuerpo", "título" y "resumen".
Sacar del análisis los registros que no tienen cuerpo o título disponible y completar los resúmenes faltantes con un campo en blanco


In [5]:
df['resumen'].fillna('', inplace=True)

In [6]:
df.dropna(inplace=True)

#### 1.2 Suplementos relevantes

Para mejorar la clasificación es conveniente retirar las secciones donde los dos diarios utilizan un vocabulario similar y muy específico del dominio como, por ejemplo, las relacionadas a deportes.

In [7]:
import numpy as np
np.unique(df['suplemento'].values)
wanted_sections = ['/cartas_al_pais/', '/economia/', '/lo-ultimo/', '/mundo/', '/opinion/',
       '/policiales/', '/politica/', '/sociedad/', 'Contratapa',
       'Economía', 'El mundo', 'El país', 'Sociedad']

In [8]:
df = df[df.suplemento.isin(wanted_sections)]

In [9]:
df = df.drop(['Unnamed: 0', 'imagen', 'fecha_hora', 'url', 'suplemento'], axis=1)

In [10]:
df.sample(2)

,cuerpo,resumen,titulo,class
80,Una de las filminas más impactantes que presen...,,Deuda: nos están avisando,1
21,El FMI prevé que América Latina y el Caribe ...,Pronosticó una caída de 10% para la economía v...,"Salvo en Venezuela, mejora la perspectiva del ...",0


#### 1.3 Corpus

Construir la columna sobre la cual vamos a predecir concatenando el título, resumen y cuerpo de las distintas noticias.

In [11]:
def build_corpus(s):
    s['corpus'] = s['cuerpo'] + s['resumen'] + s['titulo']
    return s

df = df.apply(build_corpus, axis=1)

In [12]:
df = df.drop(['cuerpo', 'resumen', 'titulo'], axis=1)

In [13]:
df.sample(5)

,class,corpus
86,1,La Asociación de Abogados Laboralistas realizó...
159,1,Muy activo en las visitas de campaña que viene...
21,0,El FMI prevé que América Latina y el Caribe ...
191,1,“¿Cuántos organismos de derechos humanos les m...
172,1,Desde Corrientes\nEl oficialismo correntino pa...


#### 1.4 Normalización del texto

Primero es conveniente pasar todo el corpus a letras minúsculas.

Para mejorar el clasificador es importante sacar todos los acentos que pueden generar diferencias artificiales entre palabras según estén o no puestos. Hint: Utilizar el módulo unidecode

Una vez retirados los acentos, quitar todos los signos de puntuación para dejar únicamente palabras. Hint: utilizar una expresión regular, por ejemplo, r'([^\s\w]|_)+' para reemplazar todo lo que no sean palabras.

In [14]:
from unidecode import unidecode
PERMITTED_CHARS = "abcdefghijklmnopqrstuvwxyz " 

def clean_string(s):
    s = s.lower()
    s = unidecode(s)
    s = "".join(c for c in s if c in PERMITTED_CHARS)
    s = " ".join(w[:4] for w in s.split(' '))
    return s

In [15]:
clean_string('“Síganlo buscando”, fue la cínica respuesta de')

'siga busc fue la cini resp de'

In [16]:
def stem_text(s):
    s['corpus'] = clean_string(s['corpus'])
    return s
    
df = df.apply(stem_text, axis=1)

In [17]:
df.sample(5)

,class,corpus
41,0,carl carr mari de mari mart garc bels aseg q...
200,1,desd coch conm de los anos de la ejec de erne...
168,1,el trib de impu de neuq decl la nuli de un all...
37,0,la cama de sena deba en comi esta sema vari ...
32,0,desp de aume las sanc puni cont los resp de ...


## 2. Modelo

Vectorizar el corpus resultante con TF-IDF y aplicar un modelo Naive Bayes con un split simple entre train y test. 
<br />
¿Cuál es la dimensión de la matriz de features? ¿Cuál es el accuracy obtenido? <br />
Dibujar la matriz de confusión.

In [18]:
X = df['corpus']
y = df['class']

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer(min_df=10)
transformer = TfidfTransformer()
vectorizer.fit(X_train)
X_train_vector = vectorizer.transform(X_train)
transformer.fit(X_train_vector)
X_train_tfidf = transformer.transform(X_train_vector)

In [21]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train_tfidf.toarray(), y_train)

GaussianNB(priors=None)

In [22]:
X_test_vector = vectorizer.transform(X_test)
X_test_tfidf = transformer.transform(X_test_vector)

In [23]:
y_predict = nb.predict(X_test_tfidf.toarray())

In [24]:
from sklearn.metrics import accuracy_score
print('Accuracy: ',accuracy_score(y_predict, y_test))

Accuracy:  0.775510204082


In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predict)

array([[12,  9],
       [ 2, 26]])

## 3. Reducción de la dimensionalidad

¿Cuál es la relación entre la cantidad de features y la cantidad de casos de entrenamiento? De acuerdo a la maldición de la dimensionalidad esta relación es un problema.<br/>
Una de las técnicas más comunes para reducir dimensiones es utilizar "stopwords", una lista con las palabras de cada idioma que no tienen un peso semántico importante. <br />
Entrenen nuevamente el modelo quitando las stopwords del español que se encuentran en el archivo "stopwords.csv". <br />
¿Qué pasa con la cantidad de dimensiones de la matriz de features? ¿Y con el accuracy?

## 4. Avanzando en la reducción de dimensiones

El modelo mejora cuando quitamos las stopwords pero todavía tenemos una cantidad de dimensiones demasiado alta para 
la cantidad de datos que tenemos.

Pensemos lo siguiente ¿Qué valor tienen las palabras que aparecen una o dos veces en todo el corpus a la hora de discriminar? Para remover del análisis las palabras que ocurren menos de determinada cantidad de veces, la clase TfidfVectorizer tiene un parámetro min_df. 

Prueben setear el parámetro min_df en 6 y volver a correr el modelo.
<br />
<strong>
¿A cuánto se reduce la dimensión de la matriz de features?
<br />
¿Cuánto mejora la performance del algoritmo?
<strong />